# Digit Recognizer

In [1]:
import pandas as pd 
import numpy as np 
import os
import cv2
import matplotlib.pyplot as plt
import warnings

from tensorflow.keras.layers import Input, Lambda, Dense, Flatten, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.applications.vgg19 import preprocess_input
from tensorflow.keras.preprocessing import image, image_dataset_from_directory
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow import keras
import tensorflow 

import scipy
print("Num GPUs Available: ", len(tensorflow.config.list_physical_devices('GPU')))


# Set the seed value for experiment reproduci.bility.
seed = 1842
tensorflow.random.set_seed(seed)
np.random.seed(seed)
# Turn off warnings for cleaner looking notebook
warnings.simplefilter('ignore')


df=pd.read_csv('train.csv')

df


Num GPUs Available:  1


,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41995,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
41996,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
41997,7,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
41998,6,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [2]:
X=df.drop('label',axis=1)
Y=df.label

In [3]:
Y=pd.get_dummies(Y,prefix='Num')

In [4]:
X.describe()

,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
count,42000.0,42000.0,42000.0,42000.0,42000.0,42000.0,42000.0,42000.0,42000.0,42000.0,...,42000.000000,42000.000000,42000.000000,42000.00000,42000.000000,42000.000000,42000.0,42000.0,42000.0,42000.0
mean,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.219286,0.117095,0.059024,0.02019,0.017238,0.002857,0.0,0.0,0.0,0.0
std,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,6.312890,4.633819,3.274488,1.75987,1.894498,0.414264,0.0,0.0,0.0,0.0
min,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.0,0.0,0.0,0.0
25%,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.0,0.0,0.0,0.0
50%,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.0,0.0,0.0,0.0
75%,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.0,0.0,0.0,0.0
max,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,254.000000,254.000000,253.000000,253.00000,254.000000,62.000000,0.0,0.0,0.0,0.0


In [5]:
X=X/255

X=X.values.reshape(-1,28,28)

In [6]:
X.shape

(42000, 28, 28)

In [7]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(X, Y, test_size=0.2, random_state=42)

In [8]:
validation_data=X_val,y_val

# ANN

İlk önce kendimize CNN kullanmadan bir mimari tasarlıyoruz.

Buradaki ayarlar hypertuning ile karşılaşırılabilir. Ben sadece iki çeşit denedim.

In [9]:
model = keras.models.Sequential([
keras.layers.Flatten(input_shape = (28,28,1)),                             
keras.layers.Dense(300, activation = 'relu' ),

keras.layers.Dense(400, activation = 'relu' ),

keras.layers.Dense(400, activation = 'relu' ),

keras.layers.Dense(400, activation = 'relu' ),
keras.layers.Dense(10, activation = 'softmax')])


model.compile(optimizer='adam',
loss=tensorflow.losses.CategoricalCrossentropy(),
metrics=[keras.metrics.AUC(name='auc')])
callback = keras.callbacks.EarlyStopping(monitor='val_loss',
                                            patience=3,
                                            restore_best_weights=True)

model.fit(X_train,y_train, epochs=20, validation_data=(validation_data), callbacks=callback)
#save model
#data manipulation 
#include graphs 

Epoch 1/20
1050/1050 [==============================] - 6s 4ms/step - loss: 0.2605 - auc: 0.9941 - val_loss: 0.1421 - val_auc: 0.9968
Epoch 2/20
1050/1050 [==============================] - 5s 5ms/step - loss: 0.1219 - auc: 0.9975 - val_loss: 0.1790 - val_auc: 0.9950
Epoch 3/20
1050/1050 [==============================] - 5s 5ms/step - loss: 0.0892 - auc: 0.9982 - val_loss: 0.1489 - val_auc: 0.9955
Epoch 4/20
1050/1050 [==============================] - 5s 5ms/step - loss: 0.0729 - auc: 0.9987 - val_loss: 0.1218 - val_auc: 0.9966
Epoch 5/20
1050/1050 [==============================] - 5s 5ms/step - loss: 0.0580 - auc: 0.9991 - val_loss: 0.1514 - val_auc: 0.9953
Epoch 6/20
1050/1050 [==============================] - 5s 4ms/step - loss: 0.0498 - auc: 0.9993 - val_loss: 0.1498 - val_auc: 0.9955
Epoch 7/20
1050/1050 [==============================] - 5s 5ms/step - loss: 0.0414 - auc: 0.9993 - val_loss: 0.1295 - val_auc: 0.9959


In [10]:
loss, accuracy = model.evaluate(X_val,y_val)
print("Loss: ", loss)
print("Accuracy: ", accuracy)

263/263 [==============================] - 1s 3ms/step - loss: 0.1218 - auc: 0.9966
Loss:  0.12182149291038513
Accuracy:  0.9965720772743225


In [11]:
#simple ann with 3 layers 
model = keras.models.Sequential([
keras.layers.experimental.preprocessing.RandomFlip('horizontal'),
keras.layers.experimental.preprocessing.RandomContrast(0.5),    

keras.layers.Flatten(input_shape = (28,28,1)),                             
keras.layers.Dense(300, activation = 'relu' ),
keras.layers.Dropout(0.1),
keras.layers.Dense(400, activation = 'relu' ),
keras.layers.Dropout(0.1),
keras.layers.Dense(400, activation = 'relu' ),
keras.layers.Dropout(0.1),
keras.layers.Dense(400, activation = 'relu' ),
keras.layers.Dense(10, activation = 'softmax')])


model.compile(optimizer='adam',
loss=tensorflow.losses.CategoricalCrossentropy(),
metrics=[keras.metrics.AUC(name='auc')])
callback = keras.callbacks.EarlyStopping(monitor='val_loss',
                                            patience=3,
                                            restore_best_weights=True)



In [12]:
model.fit(X_train,y_train, epochs=20, validation_data=(validation_data), callbacks=callback)
#save model
#data manipulation 
#include graphs 

Epoch 1/20
1050/1050 [==============================] - 7s 6ms/step - loss: 0.5183 - auc: 0.9821 - val_loss: 0.2535 - val_auc: 0.9944
Epoch 2/20
1050/1050 [==============================] - 6s 6ms/step - loss: 0.2666 - auc: 0.9936 - val_loss: 0.2121 - val_auc: 0.9954
Epoch 3/20
1050/1050 [==============================] - 6s 6ms/step - loss: 0.2109 - auc: 0.9952 - val_loss: 0.2187 - val_auc: 0.9941
Epoch 4/20
1050/1050 [==============================] - 6s 6ms/step - loss: 0.1868 - auc: 0.9962 - val_loss: 0.1744 - val_auc: 0.9962
Epoch 5/20
1050/1050 [==============================] - 6s 6ms/step - loss: 0.1591 - auc: 0.9971 - val_loss: 0.1388 - val_auc: 0.9976
Epoch 6/20
1050/1050 [==============================] - 6s 6ms/step - loss: 0.1458 - auc: 0.9971 - val_loss: 0.1459 - val_auc: 0.9966
Epoch 7/20
1050/1050 [==============================] - 6s 6ms/step - loss: 0.1357 - auc: 0.9972 - val_loss: 0.1578 - val_auc: 0.9965
Epoch 8/20
1050/1050 [==============================] - 6s 5ms

In [14]:
loss, accuracy = model.evaluate(X_val,y_val)
print("Loss: ", loss)
print("Accuracy: ", accuracy)

263/263 [==============================] - 1s 3ms/step - loss: 0.1076 - auc: 0.9974
Loss:  0.10764996707439423
Accuracy:  0.9973672032356262


# CNN

### First CNN

Burada konvolüsyon işlemi yaparak yapay sinir ağımıza farklı bir yaklaşım sergilettiriyoruz.

Üstteki gibi burada da filtrelerle oynamalar yapıp doğruluk oranının değişimine baktım.

In [15]:
#experiment with Convolutional neural net
model = keras.models.Sequential([
keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape = (28,28,1)),
keras.layers.MaxPooling2D(),
keras.layers.Conv2D(64, (2, 2), activation='relu'),
keras.layers.MaxPooling2D(),
keras.layers.Conv2D(64, (2, 2), activation='relu'),
keras.layers.Flatten(),
keras.layers.Dense(100, activation='relu'),
keras.layers.Dense(10, activation ='softmax')])

In [16]:
model.compile(optimizer='adam',
loss=tensorflow.losses.CategoricalCrossentropy(),
metrics=[keras.metrics.AUC(name='auc')])
callback = keras.callbacks.EarlyStopping(monitor='val_loss',
                                            patience=3,
                                            restore_best_weights=True)

In [17]:
model.fit(X_train,y_train, epochs=20, validation_data=validation_data, callbacks=callback)
#save model
#early stopping

Epoch 1/20
1050/1050 [==============================] - 8s 5ms/step - loss: 0.1900 - auc: 0.9969 - val_loss: 0.0640 - val_auc: 0.9989
Epoch 2/20
1050/1050 [==============================] - 5s 5ms/step - loss: 0.0536 - auc: 0.9992 - val_loss: 0.0517 - val_auc: 0.9993
Epoch 3/20
1050/1050 [==============================] - 5s 5ms/step - loss: 0.0370 - auc: 0.9996 - val_loss: 0.0568 - val_auc: 0.9987
Epoch 4/20
1050/1050 [==============================] - 5s 5ms/step - loss: 0.0289 - auc: 0.9996 - val_loss: 0.0463 - val_auc: 0.9990
Epoch 5/20
1050/1050 [==============================] - 5s 5ms/step - loss: 0.0211 - auc: 0.9998 - val_loss: 0.0471 - val_auc: 0.9988
Epoch 6/20
1050/1050 [==============================] - 5s 5ms/step - loss: 0.0183 - auc: 0.9998 - val_loss: 0.0542 - val_auc: 0.9983
Epoch 7/20
1050/1050 [==============================] - 6s 5ms/step - loss: 0.0165 - auc: 0.9999 - val_loss: 0.0379 - val_auc: 0.9991
Epoch 8/20
1050/1050 [==============================] - 5s 5ms

In [18]:
loss, accuracy = model.evaluate(X_val,y_val)
print("Loss: ", loss)
print("Accuracy: ", accuracy)

263/263 [==============================] - 1s 3ms/step - loss: 0.0379 - auc: 0.9991
Loss:  0.0379117913544178
Accuracy:  0.999081552028656


### Second CNN

In [19]:
model = keras.models.Sequential([
keras.layers.Conv2D(224, (3, 3), activation='relu', input_shape = (28,28,1)),
keras.layers.MaxPooling2D(),
keras.layers.Conv2D(448, (2, 2), activation='relu'),
keras.layers.MaxPooling2D(),
keras.layers.Conv2D(448, (2, 2), activation='relu'),
keras.layers.Flatten(),
keras.layers.Dense(100, activation='relu'),
keras.layers.Dense(10, activation ='softmax')])

In [20]:
model.compile(optimizer='adam',
loss=tensorflow.losses.CategoricalCrossentropy(),
metrics=[keras.metrics.AUC(name='auc')])
callback = keras.callbacks.EarlyStopping(monitor='val_loss',
                                            patience=3,
                                            restore_best_weights=True)

In [21]:
model.fit(X_train,y_train, epochs=20, validation_data=(validation_data), callbacks=callback)
#save model
#early stopping

Epoch 1/20
1050/1050 [==============================] - 10s 8ms/step - loss: 0.1430 - auc: 0.9975 - val_loss: 0.0537 - val_auc: 0.9994
Epoch 2/20
1050/1050 [==============================] - 9s 8ms/step - loss: 0.0488 - auc: 0.9992 - val_loss: 0.0422 - val_auc: 0.9992
Epoch 3/20
1050/1050 [==============================] - 8s 8ms/step - loss: 0.0352 - auc: 0.9995 - val_loss: 0.0311 - val_auc: 0.9996
Epoch 4/20
1050/1050 [==============================] - 9s 8ms/step - loss: 0.0271 - auc: 0.9996 - val_loss: 0.0402 - val_auc: 0.9994
Epoch 5/20
1050/1050 [==============================] - 9s 8ms/step - loss: 0.0211 - auc: 0.9998 - val_loss: 0.0470 - val_auc: 0.9989
Epoch 6/20
1050/1050 [==============================] - 9s 9ms/step - loss: 0.0163 - auc: 0.9998 - val_loss: 0.0531 - val_auc: 0.9983


In [22]:
loss, accuracy = model.evaluate(X_val,y_val)
print("Loss: ", loss)
print("Accuracy: ", accuracy)

263/263 [==============================] - 1s 4ms/step - loss: 0.0311 - auc: 0.9996
Loss:  0.031132783740758896
Accuracy:  0.9995538592338562


## Best Model

In [43]:
#experiment with Convolutional neural net
model = keras.models.Sequential([
keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape = [28,28,1]),
keras.layers.MaxPooling2D(),
keras.layers.Conv2D(32, (2, 2), activation='relu'),
keras.layers.MaxPooling2D(),

keras.layers.SeparableConv2D(64, 3, activation='relu', padding='same'),
keras.layers.SeparableConv2D(64, 3, activation='relu', padding='same'),
keras.layers.BatchNormalization(),
keras.layers.MaxPool2D(),

keras.layers.SeparableConv2D(128, 3, activation='relu', padding='same'),
keras.layers.SeparableConv2D(128, 3, activation='relu', padding='same'),
keras.layers.BatchNormalization(),
keras.layers.MaxPool2D(),
keras.layers.Dropout(0.2),



keras.layers.Flatten(),

keras.layers.Dense(512, activation='relu'),
keras.layers.BatchNormalization(),
keras.layers.Dropout(0.7),

keras.layers.Dense(128, activation='relu'),
keras.layers.BatchNormalization(),
keras.layers.Dropout(0.5),

keras.layers.Dense(64, activation='relu'),
keras.layers.BatchNormalization(),
keras.layers.Dropout(0.3),

keras.layers.Dense(10, activation ='softmax')])



In [44]:
model.compile(optimizer='adam',
loss=tensorflow.losses.CategoricalCrossentropy(),
metrics=[keras.metrics.AUC(name='auc')])

In [45]:
def exponential_decay(lr0, s):
    def exponential_decay_fn(epoch):
        return lr0 * 0.1 **(epoch / s)
    return exponential_decay_fn

exponential_decay_fn = exponential_decay(0.01, 20)

lr_scheduler = keras.callbacks.LearningRateScheduler(exponential_decay_fn)

checkpoint_cb = keras.callbacks.ModelCheckpoint("best_model.h5",
                                                    save_best_only=True)

early_stopping_cb = keras.callbacks.EarlyStopping(patience=6,restore_best_weights=True)

In [46]:
model.fit(X_train,y_train, epochs=40, validation_data=validation_data, callbacks=[checkpoint_cb, early_stopping_cb, lr_scheduler])
#save model
#early stopping

Epoch 1/40
1050/1050 [==============================] - 11s 9ms/step - loss: 0.6823 - auc: 0.9707 - val_loss: 0.4755 - val_auc: 0.9816 - lr: 0.0100
Epoch 2/40
1050/1050 [==============================] - 9s 8ms/step - loss: 0.2067 - auc: 0.9942 - val_loss: 0.1481 - val_auc: 0.9950 - lr: 0.0089
Epoch 3/40
1050/1050 [==============================] - 9s 9ms/step - loss: 0.1586 - auc: 0.9952 - val_loss: 0.1384 - val_auc: 0.9952 - lr: 0.0079
Epoch 4/40
1050/1050 [==============================] - 10s 9ms/step - loss: 0.1295 - auc: 0.9961 - val_loss: 0.1303 - val_auc: 0.9958 - lr: 0.0071
Epoch 5/40
1050/1050 [==============================] - 10s 9ms/step - loss: 0.1074 - auc: 0.9970 - val_loss: 0.0974 - val_auc: 0.9962 - lr: 0.0063
Epoch 6/40
1050/1050 [==============================] - 9s 9ms/step - loss: 0.0892 - auc: 0.9975 - val_loss: 0.0704 - val_auc: 0.9972 - lr: 0.0056
Epoch 7/40
1050/1050 [==============================] - 8s 8ms/step - loss: 0.0773 - auc: 0.9977 - val_loss: 0.0782

In [47]:
df_test=pd.read_csv('test.csv')
df_test=df_test/255

df_test=df_test.values.reshape(-1,28,28)

In [48]:
y_pred=model.predict(df_test)

875/875 [==============================] - 2s 2ms/step


In [49]:
y_pred=pd.DataFrame(np.argmax(y_pred,axis=1),columns=['Label'])

In [50]:
submission = pd.read_csv("sample_submission.csv")
submission["Label"] = y_pred
submission.to_csv('submission.csv', index=False)

In [51]:
!kaggle competitions submit -c digit-recognizer -f submission.csv -m ""

400 - Bad Request



  0%|          | 0.00/235k [00:00<?, ?B/s]
 65%|██████▍   | 152k/235k [00:00<00:00, 1.54MB/s]
100%|██████████| 235k/235k [00:01<00:00, 135kB/s] 
